# Determine if an exoplanet transit will be visible while we are in Sutherland
To determine if a star is suitable for observing from Sutherland, we need to determine if the star is above the horizon, at a reasonable airmass during the night when an exoplanet transit occurs. Fortunately, trying to observe a tranit (or eclipse of a binary star) is quite common in astronomy, so there is a python package *astroplan* which we will use to determine if a transite is visible. 

You will need to adapt this code to read in a whole series of star coordinates and names.  You will likely want simplfy the output as well.  Note that the code runs, but has one mistake I put in on purpose. Make sure you correct the mistake.

You should output a csv file, which has a column 'num_obs_transits' which gives the number of observable transits
for each star


In [6]:
import numpy as np

import astropy.units as u
from astropy.coordinates import AltAz, EarthLocation, SkyCoord
from astropy.time import Time,TimeDelta

from astroplan import Observer
from astroplan import FixedTarget
from astropy.coordinates import EarthLocation
from astropy.time import Time
from astroplan.plots import plot_airmass 
from astroplan import EclipsingSystem
from astroplan import (PrimaryEclipseConstraint, is_event_observable,
                       AtNightConstraint, AltitudeConstraint, LocalTimeConstraint)


#define our observing site

sutherland=EarthLocation.of_site('Sutherland')

#define the ra, dec, an ID name and of our target star

myra = str(172.156226)
mydec = str(-78.801587)
my_id = "TOI_1951.01"

#convert our RA and DEC into an astropy Sky Coordinate
star_coordinates = SkyCoord(myra, mydec, unit="deg")
print(star_coordinates)

#The primary eclipse is a transit for an exoplanet system. Provide the details of the orbit and transit time
# taken from the TESS csv file

primary_eclipse_time = Time(2459362.084735, format='jd')
orbital_period = 4.8294060040563 * u.day
eclipse_duration = 2.6761871714201 * u.hour  


# define where we will be observing from for astroplan
observer = Observer.at_site('Sutherland')


#let astroplan know the name and location of our target star

star1= FixedTarget(name=my_id, coord=star_coordinates)


<SkyCoord (ICRS): (ra, dec) in deg
    (172.156226, -78.801587)>


# Use astroplan to determine visibility of transit

In [3]:
#let astroplan know we have a transiting system

TOI_1951_01 = EclipsingSystem(primary_eclipse_time=primary_eclipse_time,
                           orbital_period=orbital_period, duration=eclipse_duration,
                           name=my_id)


# the start time of our observing run
obs_time = Time('2025-02-12 16:00')


#approximate number of transits which would be visible in a two week (14 day) period 

num_transits = 14.0/orbital_period.value
#convert to an integer
n_transits=round(num_transits - 0.5)
print("number transits in a 2 week period:", n_transits)


#only look for transits which occur when it is dark and occur when the star is fairly high in the sky

constraints = [AtNightConstraint.twilight_civil(),
               AltitudeConstraint(min=30*u.deg)]


#find the ingress and egress times of the transits

ing_egr = TOI_1951_01.next_primary_ingress_egress_time(obs_time, n_eclipses=n_transits)

print("All Transits:")
print(ing_egr)


# using our constraints, determine if the both the ingress and egress are observable
# the function is_event_observable returns a boolean array which lets us know which transits are observable.

can_observe=is_event_observable(constraints, observer, star1, times_ingress_egress=ing_egr)

print("Observability: ", can_observe)

num_observable = can_observe.sum()

print(num_observable, "Observable Transits of ", my_id,":")

#last part of the code will be very useful when we decide which stars we will actually try to observe in Sutherland

#in order to print out only those transit times which are observable, one needs to transpose the boolean 
#array and add an extra dimesnion to match the shape of ing_egr

can_observeT=can_observe.T
can_obs=np.insert(can_observeT,1,can_observeT[:,0],axis=1)

#print(can_obs)

#select out those transits which are observable
observe_times=ing_egr[can_obs]


if( any(observe_times) ) :
#    print("Observable Transits:")

#    convert to a more easily readable format
    t2 = Time(observe_times,format='isot')
    print(t2)
 

number transits in a 2 week period: 4
All Transits:
[[2460723.92147424 2460724.03298204]
 [2460728.75088025 2460728.86238805]
 [2460733.58028625 2460733.69179405]
 [2460738.40969226 2460738.52120006]]
Observability:  [[False False False  True]]
1 Observable Transits of  TOI_1951.01 :
['2025-03-03T21:49:57.411' '2025-03-04T00:30:31.685']
